In [1]:
import pickle

from keras.models import Model
from keras.layers import Dense, Flatten, Reshape, Input
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.optimizers import Adam

import numpy as np
from sklearn.preprocessing import LabelBinarizer

Using TensorFlow backend.


In [2]:
training_file = './data/train.p'
validation_file = './data/valid.p'
testing_file = './data/test.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_valid, y_valid = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']

In [3]:
#X_train = (X_train-128)/128
#X_valid = (X_valid-128)/128
#X_test = (X_test-128)/128

In [4]:
lb = LabelBinarizer().fit(np.hstack((y_train, y_valid, y_test)))
y_train = lb.transform(y_train)
y_valid = lb.transform(y_valid)
y_test = lb.transform(y_test)

In [5]:
_input = Input(shape=(32, 32, 3))
conv1 = Conv2D(filters=6, kernel_size=(5, 5), activation='relu', 
               strides=(1, 1), padding='valid')(_input)
pool1 = MaxPooling2D(pool_size=2, strides=2, padding='same')(conv1)
conv2 = Conv2D(filters=16, kernel_size=(5, 5), activation='relu',
               strides=(1, 1), padding='valid')(pool1)
pool2 = MaxPooling2D(pool_size=2, strides=2, padding='same')(conv2)
flattened = Flatten()(pool2)
fc1 = Dense(120, activation='relu')(flattened)
fc2 = Dense(84, activation='relu')(fc1)
output = Dense(43, activation='softmax')(fc2)

In [6]:
model = Model(inputs=[_input], outputs=[output])
model.compile(optimizer=Adam(0.001), 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 6)         456       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 6)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 10, 10, 16)        2416      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 16)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 400)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 120)               48120     
__________

In [8]:
model.fit(X_train, y_train, epochs=50, batch_size=256, 
          validation_data=(X_valid, y_valid))

Train on 34799 samples, validate on 4410 samples
Epoch 1/50
34799/34799 [==============================] - 27s - loss: 3.0434 - acc: 0.3910 - val_loss: 1.4336 - val_acc: 0.6392
Epoch 2/50
34799/34799 [==============================] - 29s - loss: 0.6522 - acc: 0.8222 - val_loss: 1.0221 - val_acc: 0.7855
Epoch 3/50
34799/34799 [==============================] - 30s - loss: 0.3177 - acc: 0.9144 - val_loss: 0.8464 - val_acc: 0.8317
Epoch 4/50
34799/34799 [==============================] - 27s - loss: 0.1938 - acc: 0.9482 - val_loss: 0.7975 - val_acc: 0.8499
Epoch 5/50
34799/34799 [==============================] - 27s - loss: 0.1392 - acc: 0.9635 - val_loss: 0.8949 - val_acc: 0.8615
Epoch 6/50
34799/34799 [==============================] - 27s - loss: 0.0971 - acc: 0.9749 - val_loss: 0.7554 - val_acc: 0.8744
Epoch 7/50
34799/34799 [==============================] - 30s - loss: 0.0808 - acc: 0.9786 - val_loss: 0.8613 - val_acc: 0.8617
Epoch 8/50
34799/34799 [==============================]

KeyboardInterrupt: 